<img src="../reports/figures/marketwatch.png" alt="Drawing" width="200">

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from selenium import webdriver 
from selenium.webdriver.common.keys import Keys   
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

<div class="alert alert-block alert-info">
    <b>Market Watch:</b>is a great websight to pull financial statements and economic reports. The goal is to use the information here
        as a 5 year trend to search for any stock on the webpage to pull in there well organized tables. 
    </div>

In [4]:
def get_financial_report(ticker):
    driver = webdriver.Firefox(executable_path= "geckodriver.exe")
    driver.maximize_window()
    driver.get('https://www.marketwatch.com/investing/stock/'+ticker+'/financials')
    timeout = 20
    ''''
    Find an ID on the page and wait before executing anything until found. After searching multiple times, 
    You will be prompted to subscribe so there was a need to close out the subscribe overlay. 
    ''''
    try:
        WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, "cx-scrim-wrapper")))
    except TimeoutException:
        driver.quit()
    subscribe = driver.find_element_by_xpath('/html/body/footer/div[2]/div/div/div[1]')
    subscribe.click()
    
    ''''
    For the income statement, balance sheet, and cashflow;  you can scrape the tables and tranpose them to 
    the years going 5 years back, for any calculations that might need to be done in the future. We kept all information
    including empty columns just in case companies had more detailed reports and information. However for the 
    most part all have the same format on market watch making it easily updateable and great for a search bar. 
    ''''

    income_table = driver.find_element_by_class_name("overflow--table").get_attribute('innerHTML')
    income  = pd.read_html(income_table)
    income_df = income[0]
    income_df.drop(columns=['5-year trend'], axis=1, inplace=True)
    income_df2 = income_df.T
    income_new_col = income_df2.iloc[0,:].values
    income_df2.columns= income_new_col
    income_df2.drop(index='Item  Item', inplace=True)
    income_df2.rename(columns={'Sales/Revenue  Sales/Revenue': 'Sales/Revenue' , 
                               'Sales Growth  Sales Growth': 'Sales Growth',
           'Cost of Goods Sold (COGS) incl. D&A  Cost of Goods Sold (COGS) incl. D&A': 'Cost of Goods Sold (COGS) incl. D&A',
           'COGS Growth  COGS Growth': 'COGS Growth', 'COGS excluding D&A  COGS excluding D&A': 'COGS excluding D&A',
           'Depreciation & Amortization Expense  Depreciation & Amortization Expense': 'Depreciation & Amortization Expense',
           'Depreciation  Depreciation': 'Depreciation',
           'Amortization of Intangibles  Amortization of Intangibles': 'Amortization of Intangibles',
           'Gross Income  Gross Income': 'Gross Income',
           'Gross Income Growth  Gross Income Growth': 'Gross Income Growth',
           'Gross Profit Margin  Gross Profit Margin': 'Gross Profit Margin',
           'SG&A Expense  SG&A Expense': 'SG&A Expense', 'SGA Growth  SGA Growth': 'SGA Growth',
           'Research & Development  Research & Development': 'Research & Development',
           'Other SG&A  Other SG&A':  'Other SG&A',
           'Other Operating Expense  Other Operating Expense': 'Other Operating Expense',
           'Unusual Expense  Unusual Expense': 'Unusual Expense',
           'EBIT after Unusual Expense  EBIT after Unusual Expense': 'EBIT after Unusual Expense',
           'Non Operating Income/Expense  Non Operating Income/Expense': 'Non Operating Income/Expense',
           'Non-Operating Interest Income  Non-Operating Interest Income': 'Non-Operating Interest Income',
           'Equity in Affiliates (Pretax)  Equity in Affiliates (Pretax)': 'Equity in Affiliates (Pretax)',
           'Interest Expense  Interest Expense': 'Interest Expense',
           'Interest Expense Growth  Interest Expense Growth': 'Interest Expense Growth',
           'Gross Interest Expense  Gross Interest Expense': 'Gross Interest Expense',
           'Interest Capitalized  Interest Capitalized': 'Interest Capitalized',
           'Pretax Income  Pretax Income': 'Pretax Income',
           'Pretax Income Growth  Pretax Income Growth': 'Pretax Income Growth',
           'Pretax Margin  Pretax Margin': 'Pretax Margin', 'Income Tax  Income Tax': 'Income Tax',
           'Income Tax - Current Domestic  Income Tax - Current Domestic': 'Income Tax - Current Domestic',
           'Income Tax - Current Foreign  Income Tax - Current Foreign': 'Income Tax - Current Foreign',
           'Income Tax - Deferred Domestic  Income Tax - Deferred Domestic': 'Income Tax - Deferred Domestic',
           'Income Tax - Deferred Foreign  Income Tax - Deferred Foreign': 'Income Tax - Deferred Foreign',
           'Income Tax Credits  Income Tax Credits': 'Income Tax Credits',
           'Equity in Affiliates  Equity in Affiliates': 'Equity in Affiliates',
           'Other After Tax Income (Expense)  Other After Tax Income (Expense)': 'Other After Tax Income (Expense)',
           'Consolidated Net Income  Consolidated Net Income': 'Consolidated Net Income',
           'Minority Interest Expense  Minority Interest Expense': 'Minority Interest Expense',
           'Net Income  Net Income': 'Net Income', 'Net Income Growth  Net Income Growth': 'Net Income Growth',
           'Net Margin Growth  Net Margin Growth': 'Net Margin Growth',
           'Extraordinaries & Discontinued Operations  Extraordinaries & Discontinued Operations': 'Extraordinaries & Discontinued Operations',
           'Extra Items & Gain/Loss Sale Of Assets  Extra Items & Gain/Loss Sale Of Assets': 'Extra Items & Gain/Loss Sale Of Assets',
           'Cumulative Effect - Accounting Chg  Cumulative Effect - Accounting Chg': 'Cumulative Effect - Accounting Chg',
           'Discontinued Operations  Discontinued Operations': 'Discontinued Operations',
           'Net Income After Extraordinaries  Net Income After Extraordinaries': 'Net Income After Extraordinaries',
           'Preferred Dividends  Preferred Dividends': 'Preferred Dividends',
           'Net Income Available to Common  Net Income Available to Common': 'Net Income Available to Common',
           'EPS (Basic)  EPS (Basic)': 'EPS (Basic)', 'EPS (Basic) Growth  EPS (Basic) Growth': 'EPS (Basic) Growth',
           'Basic Shares Outstanding  Basic Shares Outstanding': 'Basic Shares Outstanding',
           'EPS (Diluted)  EPS (Diluted)': 'EPS (Diluted)',
           'EPS (Diluted) Growth  EPS (Diluted) Growth': 'EPS (Diluted) Growth',
           'Diluted Shares Outstanding  Diluted Shares Outstanding': 'Diluted Shares Outstanding',
           'EBITDA  EBITDA': 'EBITDA', 'EBITDA Growth  EBITDA Growth': 'EBITDA Growth',
           'EBITDA Margin  EBITDA Margin': 'EBITDA Margin'}, inplace=True)
    income_df2.to_csv('../data/interim/income_statement.csv')
    
    driver.get('https://www.marketwatch.com/investing/stock/'+ticker+'/financials/balance-sheet')
    balance_sheet_table = driver.find_element_by_class_name("overflow--table").get_attribute('innerHTML')
    balance_sheet = pd.read_html(balance_sheet_table)
    balance_sheet_df = balance_sheet[0]
    balance_sheet_df.drop(columns=['5-year trend'], axis=1, inplace=True)
    balance_sheet_df2 = balance_sheet_df.T
    balance_sheet_col = balance_sheet_df2.iloc[0,:].values
    balance_sheet_df2.columns= balance_sheet_col
    balance_sheet_df2.drop(index='Item  Item', inplace=True)
    balance_sheet_df2.rename(columns={'Cash & Short Term Investments  Cash & Short Term Investments': 'Cash & Short Term Investments',
           'Cash & Short Term Investments Growth  Cash & Short Term Investments Growth': 'Short Term Investments Growth',
           'Cash Only  Cash Only': 'Cash Only',
           'Short-Term Investments  Short-Term Investments': 'Short-Term Investments',
           'Cash & ST Investments / Total Assets  Cash & ST Investments / Total Assets': 'Cash & ST Investments / Total Assets',
           'Total Accounts Receivable  Total Accounts Receivable': 'Total Accounts Receivable',
           'Total Accounts Receivable Growth  Total Accounts Receivable Growth': 'Total Accounts Receivable Growth',
           'Accounts Receivables, Net  Accounts Receivables, Net': 'Accounts Receivables, Net',
           'Accounts Receivables, Gross  Accounts Receivables, Gross': 'Accounts Receivables, Gross',
           'Bad Debt/Doubtful Accounts  Bad Debt/Doubtful Accounts': 'Bad Debt/Doubtful Accounts',
           'Other Receivable  Other Receivable': 'Other Receivable',
           'Accounts Receivable Turnover  Accounts Receivable Turnover': 'Accounts Receivable Turnover',
           'Inventories  Inventories': 'Inventories', 'Finished Goods  Finished Goods': 'Finished Goods',
           'Work in Progress  Work in Progress': 'Work in Progress', 'Raw Materials  Raw Materials': 'Raw Materials',
           'Progress Payments & Other  Progress Payments & Other': 'Progress Payments & Other',
           'Other Current Assets  Other Current Assets': 'Other Current Assets',
           'Miscellaneous Current Assets  Miscellaneous Current Assets': 'Miscellaneous Current Assets',
           'Total Current Assets  Total Current Assets': 'Total Current Assets',
           'Net Property, Plant & Equipment  Net Property, Plant & Equipment': 'Net Property, Plant & Equipment',
           'Property, Plant & Equipment - Gross  Property, Plant & Equipment - Gross': 'Property, Plant & Equipment - Gross',
           'Buildings  Buildings': 'Buildings', 'Land & Improvements  Land & Improvements': 'Land & Improvements',
           'Computer Software and Equipment  Computer Software and Equipment': 'Computer Software and Equipment',
           'Other Property, Plant & Equipment  Other Property, Plant & Equipment': 'Other Property, Plant & Equipment',
           'Accumulated Depreciation  Accumulated Depreciation': 'Accumulated Depreciation',
           'Total Investments and Advances  Total Investments and Advances': 'Total Investments and Advances',
           'Other Long-Term Investments  Other Long-Term Investments': 'Other Long-Term Investments',
           'Long-Term Note Receivables  Long-Term Note Receivables': 'Long-Term Note Receivables',
           'Intangible Assets  Intangible Assets': 'Intangible Assets', 'Net Goodwill  Net Goodwill': 'Net Goodwill',
           'Net Other Intangibles  Net Other Intangibles': 'Net Other Intangibles',
           'Other Assets  Other Assets': 'Other Assets', 'Total Assets  Total Assets': 'Total Assets',
           'Total Assets Growth  Total Assets Growth': 'Total Assets Growth'}, inplace=True)
    balance_sheet_df2.to_csv('../data/interim/balance_sheet.csv')
    
#     timeout = 20
#     # Find an ID on the page and wait before executing anything until found: 
#     try:
#         WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.ID, "cx-scrim-wrapper")))
#     except TimeoutException:
#         driver.quit()
#     subscribe = driver.find_element_by_xpath('/html/body/footer/div[2]/div/div/div[1]')
#     subscribe.click()

    driver.get('https://www.marketwatch.com/investing/stock/'+ticker+'/financials/cash-flow')
    cashflow_table = driver.find_element_by_class_name("overflow--table").get_attribute('innerHTML')
    cashflow  = pd.read_html(cashflow_table)
    cashflow_df = cashflow[0]
    cashflow_df.drop(columns=['5-year trend'], axis=1, inplace=True)
    cashflow_df2 = cashflow_df.T
    cashflow_new_col = cashflow_df2.iloc[0,:].values
    cashflow_df2.columns= cashflow_new_col
    cashflow_df2.drop(index='Item  Item', inplace=True)
    cashflow_df2.rename(columns={'Net Income before Extraordinaries  Net Income before Extraordinaries': 'Net Income before Extraordinaries',
           'Net Income Growth  Net Income Growth': 'Net Income Growth',
           'Depreciation, Depletion & Amortization  Depreciation, Depletion & Amortization': 'Depletion & Amortization',
           'Depreciation and Depletion  Depreciation and Depletion': 'Depreciation and Depletion',
           'Amortization of Intangible Assets  Amortization of Intangible Assets': 'Amortization of Intangible Assets',
           'Deferred Taxes & Investment Tax Credit  Deferred Taxes & Investment Tax Credit': 'Deferred Taxes & Investment Tax Credit' ,
           'Deferred Taxes  Deferred Taxes': 'Deferred Taxes',
           'Investment Tax Credit  Investment Tax Credit': 'Investment Tax Credit',
           'Other Funds  Other Funds': 'Other Funds' ,
           'Funds from Operations  Funds from Operations': 'Funds from Operations',
           'Extraordinaries  Extraordinaries': 'Extraordinaries',
           'Changes in Working Capital  Changes in Working Capital': 'Changes in Working Capital',
           'Receivables  Receivables': 'Receivables', 'Accounts Payable  Accounts Payable': 'Accounts Payable',
           'Other Assets/Liabilities  Other Assets/Liabilities': 'Other Assets/Liabilities',
           'Net Operating Cash Flow  Net Operating Cash Flow': 'Net Operating Cash Flow',
           'Net Operating Cash Flow Growth  Net Operating Cash Flow Growth': 'Net Operating Cash Flow Growth',
           'Net Operating Cash Flow / Sales  Net Operating Cash Flow / Sales': 'Net Operating Cash Flow / Sales'}, inplace=True)
    cashflow_df2.to_csv('../data/interim/cashflows.csv')
    driver.quit()
    return income_df2, balance_sheet_df2, cashflow_df2

In [8]:
income_statement, balance_sheet, cashflow = get_financial_report('VLO')

<div class="alert alert-block alert-info">
    <b>Income Statement</b>  is one of a company's core financial statements that shows their profit and lossProfit and Loss Statement (P&L)A profit and loss statement (P&L), or income statement or statement of operations, is a financial report that provides a summary of a over a period of time.<br><br>
    <b>Takeaway:</b>
        <dl>
            <dd><li>Net Income = (Total Revenue + Gains) – (Total Expenses + Losses)</li></dd>
            <dd><li>Total revenue is the sum of both operating and non-operating revenues while total expenses include those incurred by primary and secondary activities.</li></dd>
            <dd><li>Revenues are not receipts. Revenue is earned and reported on the income statement. Receipts (cash received or paid out) are not.</li></dd>
            <dd><li>An income statement provides valuable insights into a company’s operations, the efficiency of its management, under-performing sectors and its performance relative to industry peers.</li></dd>
         </dl>
</div>

<div class="alert alert-block alert-info">
    <b>Price-to-Earnings (P/E) Ratio</b>  Understanding earnings can be essential to determining the value of a company. To analyze earnings growth, investors can use earnings per share (EPS). This calculation measures the amount of earnings allocated to each share of stock. The P/E ratio is interpreted as a multiple, or measure, of how much an investor is paying for the stock compared to each dollar of a company’s annual earnings.
    <li><strong>Value investors</strong> are typically looking for stocks with a relatively low P/E ratio. This means they’re paying less for each dollar of earnings, which is a major characteristic of a value stock.</li>
    <dl>
        <dd>A cross-sectional analysis can provide more insight than the P/E ratio by itself. Comparing a company’s P/E ratio to a similar company or industry average can potentially reveal how well a company is actually performing. A good value stock candidate is likely to have a low P/E ratio relative to other companies in the same industry. </dd><br>
        <dd>P/E ratio referred to as a trailing P/E ratio or a forward P/E ratio. The difference is in the source of the earnings figure. A trailing P/E uses the company’s last 12 months of earnings, while a forward P/E uses an estimate of the company’s next 12 months of earnings.</dd><br>
        <dd>If a company uses aggressive accounting, the P/E ratio may appear artificially low. That’s why it can be important to also use other ratios like the price-to-sales and price-to-book ratios</dd>
    </dl>
    <li><strong>Growth investors</strong> are typically looking for stocks with a relatively high P/E ratio.</li>
    <img src="../reports/figures/pe_ratio.png" alt="Drawing" width="600"></img>
</div>

<div class="alert alert-block alert-info">
    <b>Price-to-Sales (P/S) Ratio</b>  The price-to-sales (P/S) ratio compares a stock’s price to, you guessed it, the company’s annual sales per share. This is a simple measure of how much revenue the company brings in from its sales activity. <br>
    <li><strong>Value investors</strong> are typically looking for stocks with a relatively low P/S ratio. The P/S ratio is interpreted as a multiple of how much an investor is paying for the stock compared to each dollar of a company’s annual sales.</li>
    <dl>
        <dd>One downside to using a P/S ratio is that sales numbers can often vary greatly from year-to-year or even quarter-to-quarter. Furthermore, it may seem redundant to compare price to such similar metrics as sales (P/S ratio) and earnings (P/E ratio).</dd>
    </dl>
    <img src="../reports/figures/ps_ratio.png" alt="Drawing" width="400"></img>
</div>

<div class="alert alert-block alert-info">
    <b>Price-to-Book (P/B) Ratio</b>  The P/B ratio compares the stock price to a company’s book value, which is the per-share value of shareholders’ equity.<strong> Shareholders’ equity, or book value,</strong> is a balance sheet item equal to the company’s assets minus liabilities.<br>
    <b>The book value growth rate</b>, or the percentage increase in shareholders’ equity over time, can reveal important clues about a company’s strength: an increasing book value might indicate an increase in shareholders’ equity. You typically want to see a company’s book value grow over time.<br>
    <img src="../reports/figures/bv.png" alt="Drawing" width="400"></img>
    <li><strong>Value investors</strong> generally prefer to invest in stocks with a low P/B ratio. This can indicate that they’re paying a lower multiple for the equity in the business.</li>
    <dl>
        <dd>Book value may not accurately represent the economic value of a company’s assets for two main reasons. First, book value ignores the economic value of intangible assets like brands or intellectual property. Second, book values may not reflect various accounting practices and how companies choose to record assets on their balance sheets. Despite its limitations, the P/B ratio can be a useful tool in evaluating and searching for value stocks when combined with other ratios.</dd>
    </dl>
    <img src="../reports/figures/pb_ratio.png" alt="Drawing" width="400"></img>
</div>


In [9]:
income_statement

,Sales/Revenue,Sales Growth,Cost of Goods Sold (COGS) incl. D&A,COGS Growth,COGS excluding D&A,Depreciation & Amortization Expense,Depreciation,Amortization of Intangibles,Gross Income,Gross Income Growth,...,Net Income Available to Common,EPS (Basic),EPS (Basic) Growth,Basic Shares Outstanding,EPS (Diluted),EPS (Diluted) Growth,Diluted Shares Outstanding,EBITDA,EBITDA Growth,EBITDA Margin
2015,87.8B,-,80.42B,-,78.58B,1.84B,1.3B,542M,7.39B,-,...,3.99B,8.03,-,497M,7.98,-,500M,7.73B,-,-
2016,75.66B,-13.83%,71.99B,-10.48%,70.1B,1.89B,1.32B,575M,3.67B,-50.37%,...,2.29B,4.97,-38.15%,461M,4.93,-38.18%,464M,5.59B,-27.65%,-
2017,93.98B,24.22%,89.21B,23.91%,87.22B,1.99B,1.34B,650M,4.77B,30.14%,...,4.05B,9.17,84.68%,442M,9.16,85.68%,444M,5.87B,4.92%,-
2018,117.03B,24.53%,111.39B,24.86%,109.32B,2.07B,1.4B,668M,5.64B,18.30%,...,3.11B,7.31,-20.31%,426M,7.29,-20.41%,428M,6.74B,14.93%,-
2019,108.32B,-7.45%,103.51B,-7.07%,101.26B,2.26B,1.5B,759M,4.81B,-14.83%,...,2.42B,5.85,-19.98%,413M,5.85,-19.79%,413M,6.17B,-8.45%,5.70%


<div class="alert alert-block alert-info">
    <b>Balance Sheet</b> is a financial statement that reports a company's assets, liabilities and shareholders' equity. The balance sheet is one of the three (income statement and statement of cash flows being the other two) core financial statements used to evaluate a business.<br><br>
        <b>Takeaway:</b>
            <dl>
                <dd><li>The balance sheet is one of the three (income statement and statement of cash flows being the other two) core financial statements used to evaluate a business.</li></dd>
                <dd><li>The balance sheet is a snapshot, representing the state of a company's finances (what it owns and owes) as of the date of publication.</li></dd>
                <dd><li>Fundamental analysts use balance sheets, in conjunction with other financial statements, to calculate financial ratios.</li></dd>
           </dl>
</div>

In [10]:
balance_sheet

,Cash & Short Term Investments,Short Term Investments Growth,Cash Only,Short-Term Investments,Cash & ST Investments / Total Assets,Total Accounts Receivable,Total Accounts Receivable Growth,"Accounts Receivables, Net","Accounts Receivables, Gross",Bad Debt/Doubtful Accounts,...,Accumulated Depreciation,Total Investments and Advances,Other Long-Term Investments,Long-Term Note Receivables,Intangible Assets,Net Goodwill,Net Other Intangibles,Other Assets,Total Assets,Total Assets Growth
2015,4.11B,-,4.11B,-,9.17%,4.68B,-,4.11B,4.14B,(35M),...,10.2B,201M,-,266M,156M,-,156M,2B,44.87B,-
2016,4.82B,17.06%,4.82B,-,10.28%,5.96B,27.27%,5.69B,5.72B,(32M),...,11.26B,201M,-,447M,148M,-,148M,2.11B,46.84B,4.40%
2017,5.85B,21.47%,5.85B,-,11.54%,6.92B,16.16%,6.78B,6.82B,(33M),...,12.53B,530M,-,673M,142M,-,142M,2.02B,50.69B,8.22%
2018,2.98B,-49.03%,2.98B,-,5.88%,7.35B,6.11%,6.98B,7.02B,(34M),...,13.63B,542M,-,343M,567M,260M,307M,2.18B,50.7B,0.01%
2019,2.58B,-13.38%,2.58B,-,4.74%,8.9B,21.23%,8.06B,8.09B,(36M),...,15.03B,942M,-,-,543M,260M,283M,2.82B,54.51B,7.51%


<div class="alert alert-block alert-info">
    <b>Cash Flow Statement</b> a cash flow statement, also known as statement of cash flows, is a financial statement that shows how changes in balance sheet accounts and income affect cash and cash equivalents, and breaks the analysis down to operating, investing, and financing activities.<br><br>
            <b>Takeaway:</b>
            <dl>
                <dd><li>The cash flow statement includes cash made by the business through operations, investment, and financing—the sum of which is called net cash flow.</li></dd>
                <dd><li>The first section of the cash flow statement is cash flow from operations, which includes transactions from all operational business activities.</li></dd>
                <dd><li>Cash flow from investment is the second section of the cash flow statement, and is the result of investment gains and losses.</li></dd>
                 <dd><li>Cash flow from financing is the final section, which provides an overview of cash used from debt and equity.</li></dd>
           </dl>
</div>

In [11]:
cashflow

,Net Income before Extraordinaries,Net Income Growth,Depletion & Amortization,Depreciation and Depletion,Amortization of Intangible Assets,Deferred Taxes & Investment Tax Credit,Deferred Taxes,Investment Tax Credit,Other Funds,Funds from Operations,Extraordinaries,Changes in Working Capital,Receivables,Accounts Payable,Other Assets/Liabilities,Net Operating Cash Flow,Net Operating Cash Flow Growth,Net Operating Cash Flow / Sales
2015,4.1B,-,1.84B,1.3B,542M,165M,165M,-,790M,6.9B,-,(1.29B),1.19B,(1.79B),(100M),5.61B,-,6.39%
2016,2.42B,-41.06%,1.89B,1.32B,575M,230M,230M,-,(691M),3.85B,-,970M,(1.38B),1.56B,(33M),4.82B,-14.10%,6.37%
2017,4.16B,71.95%,1.99B,1.34B,650M,(2.54B),(2.54B),-,-,3.6B,-,1.88B,(870M),1.84B,917M,5.48B,13.73%,5.83%
2018,3.35B,-19.32%,2.07B,1.4B,668M,203M,203M,-,-,5.63B,-,(1.25B),(457M),304M,(107M),4.37B,-20.27%,3.73%
2019,2.78B,-16.97%,2.26B,1.5B,759M,234M,234M,-,-,5.27B,-,258M,(1.47B),1.53B,451M,5.53B,26.54%,5.11%


<div class="alert alert-block alert-success">
    <b>income_statement, balance_sheet, cashflow = get_financial_report('VLO')</b> is a function that 
    generates these reports through a ticker input. A 5 year report to search for any stock on the webpage to pull in there well organized tables. 
    </div>

# Fundamental Analysis

<div class="alert alert-block alert-info">
    <b>Fundamental Analysis:</b> is an approach to identifying investment opportunities. Fundamental investors use financial statements, economic reports, and forecasts to examine the key drivers of a company’s current and future potential business activities.<br>
    <br>
    <b>Fundamental Investors:</b> Fundamental investors often take a holistic approach by examining all of the company’s <strong>strengths, weaknesses, opportunities, and threats</strong>. These items form the basis of an investment thesis, and uncover areas that may require additional analysis.<br>
    <br>
    <li><strong>Strengths</strong> are areas that the business does well, like a company with high net profit margins.</li>
    <li><strong>Weaknesses</strong> are areas that the business doesn’t do well, like carrying too much debt.</li>
    <li><strong>Opportunities</strong> are areas that can likely be improved rather quickly.</li>
     <li><strong>Threats</strong> are areas where competitors are potentially able to outperform a company, like with a newly developed
         product.</li><br>
    <br>
     <b>Long-Term Business Trends:</b> less influenced by short-term noise. For example, imagine a company in an up-and-coming industry that has secured a 10-year patent on a revolutionary technology.<br>
    <br>
    <b>Assumptions:</b> estimating a stock’s intrinsic value or future growth potential requires making assumptions. But a stock’s market value may never catch up with the value you determine.<br>
     <br>
    <b>Forecasting Errors:</b> An error can come from many sources, including inaccurate data, accounting blunders, or simply mathematical mistakes.
    </div>

<div class="alert alert-block alert-danger">
    <b>get_financial_reports(ticker):</b> The original function was meant to just use the following libraries<strong> requests, bs4, and datetime,</strong> however, it became apparent if any alteration was done to the site, it didn't seem proficent compared to using something like <strong>Selenium.</strong>
</div>

In [ ]:
import requests
from datetime import *
from bs4 import BeautifulSoup as bs

In [ ]:
def get_financial_reports(ticker):
#build URLs
    urlfinancials = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials'
    urlbalancesheet = 'https://www.marketwatch.com/investing/stock/'+ticker+'/financials/balance-sheet'
#request the data using beautiful soup
    text_soup_financials = BeautifulSoup(requests.get(urlfinancials).text,"html") 
    text_soup_balancesheet = BeautifulSoup(requests.get(urlbalancesheet).text,"html")
    # build lists for Income statement
    titlesfinancials = text_soup_financials.findAll('td', {'class': 'rowTitle'})
    epslist=[]
    netincomelist = []
    longtermdebtlist = [] 
    interestexpenselist = []
    ebitdalist= []
    #load data into lists if the row title is found
    for title in titlesfinancials:
        if 'EPS (Basic)' in title.text:
            epslist.append ([td.text for td in  title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Net Income' in title.text:
            netincomelist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Interest Expense' in title.text:
            interestexpenselist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'EBITDA' in title.text:
            ebitdalist.append ([td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
    # find the table headers for the Balance sheet
    titlesbalancesheet = text_soup_balancesheet.findAll('td', {'class': 'rowTitle'})
    equitylist=[]
    for title in titlesbalancesheet:
        if 'Total Shareholders\' Equity' in title.text:
            equitylist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
        if 'Long-Term Debt' in title.text:
            longtermdebtlist.append( [td.text for td in title.findNextSiblings(attrs={'class': 'valueCell'}) if td.text])
    #get the data from the income statement lists 
    #use helper function get_element
    eps = get_element(epslist,0)
    epsGrowth = get_element(epslist,1)
    netIncome = get_element(netincomelist,0)
    shareholderEquity = get_element(equitylist,0)
    roa = get_element(equitylist,1)
    longtermDebt = get_element(longtermdebtlist,0)
    interestExpense =  get_element(interestexpenselist,0)
    ebitda = get_element(ebitdalist,0)

    # load all the data into dataframe 
    fin_df= pd.DataFrame({'eps': eps,'eps Growth': epsGrowth,'net Income': netIncome,'shareholder Equity': shareholderEquity,'roa': roa,'longterm Debt': longtermDebt,'interest Expense': interestExpense,'ebitda': ebitda}, index=range(date.today().year-5,date.today().year))

    fin_df.reset_index(inplace=True)

    return fin_df

In [ ]:
#helper function
def get_element(list,element):
    try:
        return list[element]
    except:
        return '-'         